In [ ]:
import requests
import pandas as pd
import time

# Load wallet addresses from Excel
wallets_df = pd.read_excel(r"C:\Users\Anjani Kumar\Downloads\Wallet id.xlsx")  # Make sure this file exists in the same folder
wallets = wallets_df['wallet_id'].str.lower().dropna().unique().tolist()

API_KEY = "API KEY HERE"  # Replace with your Covalent API key

# Example Compound V2 token contracts
compound_contracts = [
    "0x5d3a536e4d6dbd6114cc1ead35777bab948e3643",  # cDAI
    "0x39aa39c021dfbae8fac545936693ac917d5e7563",  # cUSDC
    "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",  # cETH
    "0xccf4429db6322d5c611ee964527d42e5d685dd6a",  # cWBTC
]

def fetch_transactions(wallet, contract=None):
    base_url = f"https://api.covalenthq.com/v1/1/address/{wallet}/transfers_v2/"
    params = {
        "key": API_KEY,
        "contract-address": contract,
        "page-size": 1000
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return pd.json_normalize(data['data']['items'])
    else:
        print(f"Failed for {wallet} | Status: {response.status_code}")
        return pd.DataFrame()

# Fetch all transactions
all_txns = []

for wallet in wallets:
    for contract in compound_contracts:
        df = fetch_transactions(wallet, contract)
        if not df.empty:
            df['wallet'] = wallet
            all_txns.append(df)
        time.sleep(1.2)

# Save to CSV
if all_txns:
    final_df = pd.concat(all_txns, ignore_index=True)
    final_df.to_csv("compound_wallet_transactions.csv", index=False)
    print("✅ Saved to 'compound_wallet_transactions.csv'")
else:
    print("❌ No data retrieved.")

✅ Saved to 'compound_wallet_transactions.csv'


In [16]:
import pandas as pd
import numpy as np

# Load transaction data
df = pd.read_csv("compound_wallet_transactions.csv")

# Convert timestamp to datetime if available
if 'block_signed_at' in df.columns:
    df['timestamp'] = pd.to_datetime(df['block_signed_at'])

# Normalize token transfer values (convert raw values using token decimals)
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df = df.dropna(subset=['value'])

# Group-level features per wallet
features = df.groupby('wallet').agg({
    'value': ['sum', 'mean', 'max', 'count'],
    'tx_hash': 'nunique',
    'timestamp': ['min', 'max'],
}).reset_index()

# Flatten MultiIndex column names
features.columns = ['_'.join(col).strip('_') for col in features.columns.values]

# Rename for clarity
features.rename(columns={
    'wallet_': 'wallet',
    'value_sum': 'total_value_transferred',
    'value_mean': 'avg_tx_value',
    'value_max': 'max_tx_value',
    'value_count': 'tx_count',
    'tx_hash_nunique': 'unique_txns',
    'timestamp_min': 'first_tx_time',
    'timestamp_max': 'last_tx_time',
}, inplace=True)

# Time-related features
features['activity_span_days'] = (
    pd.to_datetime(features['last_tx_time']) - pd.to_datetime(features['first_tx_time'])
).dt.days + 1

# Fill any missing or zero activity spans
features['activity_span_days'] = features['activity_span_days'].replace(0, 1)

# Normalize transactional intensity
features['tx_per_day'] = features['tx_count'] / features['activity_span_days']

# Drop time fields if not needed
features.drop(['first_tx_time', 'last_tx_time'], axis=1, inplace=True)

# Save feature set
features.to_csv("wallet_features.csv", index=False)
print("✅ Features saved to 'wallet_features.csv'")


✅ Features saved to 'wallet_features.csv'


In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import numpy as np

# Load wallet features
df = pd.read_csv("wallet_features.csv")

# Keep wallet ID aside
wallets = df['wallet']
X = df.drop(columns=['wallet'])

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Convert scaled data back to DataFrame for clustering logic
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# KMeans for proxy labeling (unsupervised learning)
kmeans = KMeans(n_clusters=2, random_state=42)
proxy_labels = kmeans.fit_predict(X_scaled_df)

# Identify safer cluster using a meaningful column
# Choose the best available column from your features
possible_safe_features = ['avg_tx_value', 'tx_count', 'total_value_transferred', 'tx_per_day']
available_columns = X_scaled_df.columns.tolist()

# Pick the first matching safe feature
for col in possible_safe_features:
    if col in available_columns:
        safe_column = col
        break
else:
    # Default to first column if none of the known ones are found
    safe_column = available_columns[0]

# Group stats by cluster and find which is safer (higher value of the chosen feature)
cluster_stats = X_scaled_df.groupby(proxy_labels).mean()
safe_cluster = cluster_stats[safe_column].idxmax()

# Assign binary label: 1 = safe, 0 = risky
y = (proxy_labels == safe_cluster).astype(int)

# Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_scaled, y)

# Get probability of being safe
probs = rf.predict_proba(X_scaled)[:, 1]

# Scale to 0–1000 as final score
credit_scores = (probs * 1000).astype(int)

# Final result
result_df = pd.DataFrame({
    'wallet_id': wallets,
    'score': credit_scores
})

# Save to CSV
result_df.to_csv("wallet_risk_scores.csv", index=False)
print("✅ Saved risk scores to 'wallet_risk_scores.csv'")


✅ Saved risk scores to 'wallet_risk_scores.csv'
